<a href="https://colab.research.google.com/github/aaaksenova/RussianConstructicon_NLP_tools/blob/change/Definitions_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Статистика

Подсчет поверхностной статистики по текущей версии конструктикона. Распределение средней длин определений по семаническим и синтаксическим типам, частиречный анализ.

By Екатерина Волошина (vocat@mail.ru)

In [ ]:
from google.colab import auth, files
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
cred = GoogleCredentials.get_application_default()
gc = gspread.authorize(cred)
from string import punctuation
from collections import Counter
import nltk
from nltk.util import trigrams
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import WordPunctTokenizer, PunktSentenceTokenizer
import matplotlib.pyplot as plt


russian_stopwords = stopwords.words("russian")

In [ ]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1dJNFooYJuquQaLm7I0s0DT5mekgVEfUtM128SFARsVU')
sheet = wb.worksheet('Master')

In [ ]:
df = pd.DataFrame(sheet.get_all_values())

In [ ]:
df.columns = df.iloc[0]
df = df.iloc[2:]

In [ ]:
df['Synt. function'] = df['Synt. function'].replace('Biclausal/Conj ', 'Biclausal / Conj')
df['Synt. function'] = df['Synt. function'].replace('Complex prep', 'Complex Prep')
df['Synt. function'] = df['Synt. function'].replace('Clause ', 'Clause')
df['Synt. function'] = df['Synt. function'].replace(['PP; Simplex prep', 'PP, simplex preposition', 'PP; simplex prep'], 'PP, Simplex Prep')
df['Synt. function'] = df['Synt. function'].replace('Nominal pattern; PP; SImplex prep; Noun', 'Nominal pattern; PP; Simplex prep; Noun')
df['Synt. function'] = df['Synt. function'].replace('Nominal pattern/Complex Prep', 'Nominal Pattern/Complex Prep')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()
tokenizer = WordPunctTokenizer()
sent_tokenizer = PunktSentenceTokenizer()
punctuation = punctuation + ', ' + ' [' + '\n' + '. '

In [ ]:
df['Tokenized definition'] = [[n for n in tokenizer.tokenize(i) if n not in punctuation] for i in df['Definition in Russian']]
df['Sentences'] = [[s for s in sent_tokenizer.tokenize(i)] for i in df['Definition in Russian']]
lengths = [len(d) for d in df['Tokenized definition']]
sent_lengts = [len(s) for s in df['Sentences']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
def pos_tag(sentence):
    tags = []
    for i in sentence:
        tags.append(str(morph.parse(i)[0].tag.POS))
    return [i for i in tags if i != 'None']

In [ ]:
df['POS'] = [pos_tag(i) for i in df['Tokenized definition']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
print(f'Средняя длина определений: {sum(lengths)/len(lengths)}')

Средняя длина определений: 20.150442477876105


In [ ]:
print(f'Среднее количество предложений - {sum(sent_lengts)/len(sent_lengts)}')

Среднее количество предложений - 1.4336283185840708


In [ ]:
df = df.drop(index=df[df['SemType1'] == '#N/A'].index)

In [ ]:
def mean_len():
    values = sorted(set(df['SemType1']))
    syn_values = sorted(set(df['Synt. function']))
    print(syn_values)
    mean_length = []
    sen_length = []
    synt_values = []
    word_length = []
    got_values = []
    num = []
    verbs = []
    ngram = []
    for i in tqdm(values):
        for a in tqdm(syn_values):
            length = [len(d) for d in df[(df['SemType1'] == i)&(df['Synt. function'] == a)]['Tokenized definition']]
            if sum(length) != 0:
                sentence = [len(s) for s in df[(df['SemType1'] == i) & (df['Synt. function'] == a)]['Sentences']]
                w_p_s = [[len([n for n in tokenizer.tokenize(i) if n not in punctuation]) for i in s] for s in df[(df['SemType1'] == i) & (df['Synt. function'] == a)]['Sentences']]
                verb = [i.count('VERB') for i in df[(df['SemType1'] == i) & (df['Synt. function'] == a)]['POS']]
                pos_ngrams = list(trigrams([n for i in df[(df['SemType1'] == i) & (df['Synt. function'] == a)]['POS'] for n in i if n != 'None']))
                c = Counter(pos_ngrams)
                ngram.append(' ,'.join(' + '.join(i[0]) for i in c.most_common()[:3]))
                tokens = [i for s in w_p_s for i in s]
                mean_length.append(sum(length)/len(length))
                sen_length.append(sum(sentence)/len(sentence))
                word_length.append(sum(tokens)/len(tokens))
                got_values.append(i)
                synt_values.append(a)
                num.append(len(length))
                verbs.append(sum(verb)/len(verb))
    return got_values, synt_values, mean_length, sen_length, word_length, num, verbs, ngram

In [ ]:
df_all = pd.DataFrame(columns=['Sem type', 'Syn type', 'Length', 'Sentences', 'Sentence length', 'Number of definitions', 'Verbs', 'POS Ngrams'])

In [ ]:
got_values, syn_values, mean_length, sen_length, word_length, num, verbs, ngram = mean_len()

['', 'Adj', 'Adv', 'Adv; Prep', 'Biclausal', 'Biclausal / Conj', 'Clause', 'Complex Prep', 'Complex prep / Nominal pattern', 'Conj', 'Conj; Adv', 'Conj; Negation', 'Conj; Reduplication', 'Conj; Verb', 'Double negation; Adv', 'Matrix', 'N/A', 'Nominal Pattern/Complex Prep', 'Nominal pattern', 'Nominal pattern; PP; Simplex prep; Noun', 'Nominal pattern; Reduplication', 'Noun; Adj', 'Noun; Negation; Simplex prep', 'Noun; PP; simplex prep', 'Noun; Simplex prep', 'Noun; Simplex prep; Pron; Conj', 'PP', 'PP, Num', 'PP, Simplex Prep', 'PP, Simplex prep', 'PP; negation', 'PP; simplex prep; Noun', 'Parenth', 'Prep', 'Reduplication', 'Reduplication; Nominal pattern; Simplex prep', 'Reduplication; Simplex prep; Conj; Adv', 'Simplex prep', 'Simplex prep; Conj; PP', 'Simplex prep; Nominal pattern', 'VP; Adj', 'VP; Conj', 'VP; PP; simplex prep', 'Verb', 'Verb pattern', 'Verb pattern; Adv; Double negation']


In [ ]:
df_all['Sem type'] = got_values
df_all['Syn type'] = syn_values
df_all['Length'] = mean_length
df_all['Sentences'] = sen_length
df_all['Sentence length'] = word_length
df_all['Number of definitions'] = num
df_all['Verbs'] = verbs
df_all['POS Ngrams'] = ngram

In [ ]:
df_all[df_all['Number of definitions'] > 2].sort_values('Sentence length', ascending=False)[:50]

,Sem type,Syn type,Length,Sentences,Sentence length,Number of definitions,Verbs,POS Ngrams
40,Instrument,Simplex prep,24.500000,1.250000,19.600000,4,2.750000,"ADJF + CONJ + NOUN ,NOUN + PREP + NOUN ,NOUN +..."
29,Condition,Biclausal / Conj,51.000000,2.666667,19.125000,3,6.000000,"NOUN + CONJ + NOUN ,NOUN + PREP + NOUN ,CONJ +..."
60,Non-standard subject,,5.428571,0.285714,19.000000,7,0.714286,"NOUN + VERB + CONJ ,NOUN + CONJ + PREP ,CONJ +..."
86,Spatial expression,Nominal pattern,18.333333,1.000000,18.333333,3,1.333333,"NOUN + NOUN + VERB ,NOUN + VERB + NOUN ,VERB +..."
17,Caritive,,36.600000,2.000000,18.300000,5,4.200000,"NOUN + CONJ + NOUN ,NOUN + VERB + CONJ ,ADJF +..."
2,Addressee,,21.166667,1.166667,18.142857,6,2.000000,"VERB + ADJF + NOUN ,ADJF + NOUN + PREP ,PREP +..."
26,Condition,,25.605263,1.421053,18.000000,38,2.710526,"NOUN + NOUN + VERB ,NOUN + PREP + NOUN ,ADJF +..."
71,Quantification,,15.888889,0.888889,17.875000,9,1.777778,"NOUN + VERB + CONJ ,NOUN + NOUN + VERB ,CONJ +..."
84,Spatial expression,,25.333333,1.500000,16.888889,6,2.666667,"PREP + ADJF + NOUN ,NOUN + VERB + CONJ ,ADJF +..."
15,Calculation,,16.000000,1.000000,16.000000,14,1.000000,"NOUN + ADJF + NOUN ,ADJF + NOUN + NOUN ,NOUN +..."


In [ ]:
pos_text = [n for i in df['POS'] for n in i]

In [ ]:
tokens = list(trigrams(pos_text))
c = Counter(tokens)

In [ ]:
for i in c.most_common()[:20]:
    print(' + '.join(i[0]), '\t', i[1])

NOUN + NOUN + VERB 	 349
ADJF + NOUN + NOUN 	 249
NOUN + NOUN + NOUN 	 207
PREP + NOUN + NOUN 	 201
NOUN + CONJ + NOUN 	 195
PREP + ADJF + NOUN 	 178
NOUN + VERB + CONJ 	 177
NOUN + ADJF + NOUN 	 163
NOUN + PREP + NOUN 	 163
CONJ + NOUN + NOUN 	 159
NOUN + VERB + PREP 	 144
VERB + PREP + NOUN 	 124
ADJF + CONJ + NOUN 	 119
NOUN + PREP + ADJF 	 117
VERB + ADJF + NOUN 	 109
NOUN + NOUN + CONJ 	 108
NOUN + VERB + ADJF 	 105
ADJF + NOUN + CONJ 	 103
PREP + NOUN + ADJF 	 92
NOUN + VERB + NOUN 	 89
